In [1]:
import numpy as np
import pandas as pd
import requests
import utils

# Sensitivity to eutrophication

# Part 2: Waterbody properties

Scraping additional information for each waterbody from Vann-Nett. See e-mail from Sonja K received 21.01.2025 for details.

## 1. Waterbodies of interest

In [2]:
# Read WB list from MDir
xl_path = r"../data/mdir_waterbodies_list.xlsx"
wb_df = pd.read_excel(xl_path)

# Strip leading and trailing whitespace
wb_df["VannforekomstID"] = wb_df["VannforekomstID"].str.strip()

# Check unique
print("The following waterbodies are duplicated. Only the last will be kept:")
display(wb_df[wb_df.duplicated("VannforekomstID", keep=False)])
wb_df.drop_duplicates("VannforekomstID", keep="last", inplace=True)
wb_df = wb_df.dropna(subset="VannforekomstID").reset_index(drop=True)

The following waterbodies are duplicated. Only the last will be kept:


,VannforekomstID,Id,Navn.vannforekomst,Breddegrad,Lengdegrad,Zero,Areal (km2),Type,Opphold.bunnvann,NIVA.data,oppdrag
141,050-106-R,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,Hovedoppdrag
142,050-105-R,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,Hovedoppdrag
153,050-105-R,R01,Bjoreio overføring fra Sysenvatnet - dam Tveito,60.4217,7.2528,0.0,17.369,R,NaN,NaN,NaN
154,050-106-R,R02,Isdølo Isdalsvatnet - Bjoreio,60.4302,7.2762,0.0,3.356,R,NaN,Vannforekomsten oppstrøms er med i referanseel...,NaN


In [3]:
# Get details from Vann-Nett
wb_list = wb_df["VannforekomstID"].tolist()
reg_list = []
vassom_no_list = []
vassom_name_list = []
sal_list = []
type_list = []
o2_vals = []
o2_units = []
o2_samps = []
o2_from_yrs = []
o2_to_yrs = []

for wb_id in wb_list:
    # Get basic WB info
    url = f"https://vann-nett.no/service/waterbody/{wb_id}"
    response = requests.get(url)
    if response.status_code != 200:
        response.raise_for_status()
    data = response.json()

    # Extract relevant data
    reg_id = data["generalInformation"]["catchmentId"]
    vassom_no = data["generalInformation"]["riverBasin"]["id"]
    vassom_name = data["generalInformation"]["riverBasin"]["text"]
    try:
        sal = data["waterType"]["typeParameters"]["salinity"]["text"]
    except KeyError:
        sal = np.nan
    try:
        water_type = data["waterType"]["nationalWaterType"]
    except KeyError:
        water_type = np.nan

    # Get oxygen data
    eco_df = utils.get_data_from_vannnett(wb_id, 'ecological')
    if eco_df is not None:
        eco_df = eco_df.query("parameter == 'Dissolved oxygen'")
        assert len(eco_df) <= 1
        if len(eco_df) == 1:
            o2_vals.append(eco_df['value'].iloc[0])
            o2_units.append(eco_df['unit'].iloc[0])
            o2_samps.append(eco_df['sample_count'].iloc[0])
            o2_from_yrs.append(eco_df['year_from'].iloc[0])
            o2_to_yrs.append(eco_df['year_to'].iloc[0])
        else:
            o2_vals.append(np.nan)
            o2_units.append(np.nan)
            o2_samps.append(np.nan)
            o2_from_yrs.append(np.nan)
            o2_to_yrs.append(np.nan)
    else:
        o2_vals.append(np.nan)
        o2_units.append(np.nan)
        o2_samps.append(np.nan)
        o2_from_yrs.append(np.nan)
        o2_to_yrs.append(np.nan)

    reg_list.append(reg_id)
    vassom_no_list.append(vassom_no)
    vassom_name_list.append(vassom_name)
    sal_list.append(sal)
    type_list.append(water_type)

wb_df["regine"] = reg_list
wb_df["vassom_no"] = vassom_no_list
wb_df["vassom_name"] = vassom_name_list
wb_df["national_water_type"] = type_list
wb_df["salinity_class"] = sal_list
wb_df["O2_value"] = o2_vals
wb_df["O2_unit"] = o2_units
wb_df["O2_sample_count"] = o2_samps
wb_df["O2_from_year"] = o2_from_yrs
wb_df["O2_to_year"] = o2_to_yrs

out_xl_path = r"../data/mdir_waterbodies_with_vann-nett_info.xlsx"
wb_df.to_excel(out_xl_path, index=False)